## Notebook to load catalog data, and perform the iteration

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import astropy.units as u
import gc
from astropy.coordinates import SkyCoord, GeocentricTrueEcliptic
from library.snr import optimal_snr, optimal_snr_AE
from library.lisa_psd import noise_psd_AE

/home/alice/miniforge3/envs/gwpy_env/lib/python3.12/site-packages/pycbc/types/array.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal as _lal
PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


## Step 1: pre-process the catalog

In the terminal, run:

`python preprocess_catalog.py     --filepath 
   --output     --T_obs      --delta_t      --tdi 1.5 or 2.0     --snr_preselection       --batch_size      --keys `


default parameters: T_obs = 126144000 (4 years), delta_t = 5, tdi = 1.5, snr_preselection = 0.01, batch_size = 1000, keys = ["GW22FrequencySourceFrame", "GW22FrequencyDerivativeSourceFrame",
                "Amplitude", "Declination", "RightAscension", "PolarisationAngle",
                "InclinationAngle", "SecondaryMassSSBFrame", "PrimaryMassSSBFrame",
                "TotalMassSSBFrame"]


## Step 2: load processed data 

In [2]:
from library.iteration_utils import load_waveforms, setup, run_iterative_separation

data = load_waveforms('processed_all_snr000_1yrs_tdi15.h5')

## Step 3: prepare state and parameters for the iteration and run it

Set SNR threshold and name of the output file with the resolved sources data. In `run_iterative_separation`, you can set the maximum number of iterations and the size of the median filter for the smoothing of the PSD, as well as turn on/off plots and saving the results.

In [ ]:
snr_thr = 7
results_filename = 'results_all_snr7_1yrs_sel01_tdi15'
T_obs = data['T_obs']
max_iterations = 50
tdi = 1.5
filter_size = 2000

state = setup(data, snr_calculator= lambda source: optimal_snr_AE(source["A"], source["E"], source["psd_total"], T_obs=T_obs), 
              psd_instrumental=noise_psd_AE, 
              snr_threshold=snr_thr,
              tdi = tdi,
              filter_size=filter_size)
del data
gc.collect()
results = run_iterative_separation(state,  
                                   max_iterations=max_iterations, 
                                   filter_size=filter_size, 
                                   print_progress=True, 
                                   plot= False, 
                                   save_results=True, 
                                   output_file= results_filename)

Starting with 15539324 total sources
15514455 sources already pre-excluded and added to the confusion
24869 candidate sources to run the iteration
Frequency range: 3.49e-04 to 2.04e-02 Hz
SNR threshold: 7

--- Step 0: Instrument-only SNR pre-exclusion
  Unresolvable sources (added to confusion): 11541
  SNR candidates for iteration: 13328

--- Iteration 1 ---
  New resolved: 2582
  Still unresolved: 10746
  Total resolved: 2582

--- Iteration 2 ---
  New resolved: 772
  Still unresolved: 9974
  Total resolved: 3354

--- Iteration 3 ---
  New resolved: 234
  Still unresolved: 9740
  Total resolved: 3588

--- Iteration 4 ---
  New resolved: 92
  Still unresolved: 9648
  Total resolved: 3680

--- Iteration 5 ---
  New resolved: 30
  Still unresolved: 9618
  Total resolved: 3710

--- Iteration 6 ---
  New resolved: 2
  Still unresolved: 9616
  Total resolved: 3712

--- Iteration 7 ---
  New resolved: 1
  Still unresolved: 9615
  Total resolved: 3713

--- Iteration 8 ---
  New resolved: 1
 